In [ ]:
!pip install qiskit
!pip install qiskit-aer
#These lines install Qiskit, a Python library for working with quantum computers,
#and the Aer package, which simulates quantum circuits on a classical computer.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 40.3 MB/s eta 0:00:00


In [ ]:

from qiskit import QuantumCircuit, transpile
#QuantumCircuit: Used to create and manipulate quantum circuits.
#transpile: Converts a quantum circuit to one that can be run on a specific backend (simulator or actual quantum hardware).


In [ ]:
from qiskit_aer import AerSimulator
#AerSimulator: A simulator that mimics the behavior of a quantum computer.

In [ ]:
from qiskit.visualization import plot_histogram
#plot_histogram: Visualizes the results of quantum circuits.

In [ ]:
import random
#random: A standard Python module used to generate random bits and bases.

In [ ]:
def generate_random_bits(length):
    return [random.randint(0, 1) for _ in range(length)]
#This function generates a list of random bits (0 or 1) with the specified length.
# It's for Alice to create a random sequence of bits to encode the key.

In [ ]:
def generate_random_bases(length):
    return [random.choice(['Z', 'X']) for _ in range(length)]
#This function generates a list of random bases ('Z' or 'X') for each bit. 'Z' represents the computational basis,
#and 'X' represents the diagonal basis. Both Alice and Bob randomly choose these bases.

In [ ]:
def encode_bits(bits, bases):
    qubits = []
    for bit, base in zip(bits, bases):
        qc = QuantumCircuit(1, 1)
        if bit == 1:
            qc.x(0)
        if base == 'X':
            qc.h(0)
        qubits.append(qc)
    return qubits
#This function creates quantum circuits based on the random bits and bases:
#If the bit is 1, Alice flips the qubit using the X gate.
#If the base is 'X', she applies the Hadamard gate, which puts the qubit into a superposition state.
#These qubits represent Alice's encoded key.

In [ ]:
def measure_qubits(qubits, bases):
    bits = []
    for qc, base in zip(qubits, bases):
        if base == 'X':
            qc.h(0)
        qc.measure(0, 0)
        simulator = AerSimulator()
        compiled_circuit = transpile(qc, simulator)
        result = simulator.run(compiled_circuit, shots=1).result()
        counts = result.get_counts(qc)
        measured_bit = int(list(counts.keys())[0])
        bits.append(measured_bit)
    return bits
#Bob measures the qubits he receives using his randomly chosen bases.
#If Bob’s base is 'X', he applies the Hadamard gate before measurement to match Alice's encoding.

In [ ]:
def bb84_protocol(key_length):
    alice_bits = generate_random_bits(key_length)
    alice_bases = generate_random_bases(key_length)

    qubits = encode_bits(alice_bits, alice_bases)
    bob_bases = generate_random_bases(key_length)
    bob_bits = measure_qubits(qubits, bob_bases)
    shared_key = []
    for i in range(key_length):
        if alice_bases[i] == bob_bases[i]:
            shared_key.append(alice_bits[i])

    return shared_key
#Alice generates random bits and bases, encodes her bits into qubits, and sends them to Bob.
#Bob randomly chooses his own bases and measures the qubits.
#Key Matching: If Alice's base matches Bob's base at a certain position, the bit at that position is used in the final shared key.

In [ ]:
key_length = 70
shared_key = bb84_protocol(key_length)
print("Shared Key:", shared_key)
#A key of length is generated and printed,
#This is the final shared key that both Alice and Bob agreed on after comparing their bases.
#Only the bits where Alice's and Bob's bases matched are included in the final key.


Shared Key: [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1]
